In [1]:
# magics: ensures that any changes to the modules loaded below will be re-loaded automatically
%load_ext autoreload
%autoreload 2

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import copy
import scipy.optimize as optimize
import time

from newtest import child_model
import NPL
from Solve_test import solve_NFXP
import estimate_test as estimate


In [7]:
model = child_model()
model.mu


-0.12

Simulate data with solved model

In [8]:
# Simulate the data
# Set up

solver = solve_NFXP()


# update starting value: 
ev0 = np.zeros((model.n))
ev,pk= solver.poly(model.bellman, ev0, beta = model.beta, output=2)

# data
data = model.sim_data(pk) 
samplesize = data.shape[0]

In [9]:
tabulate = data.dx1.value_counts()
tabulate
#[tabulate[i]/sum(tabulate) for i in range(tabulate.size-1)]


0    14362
1     5638
Name: dx1, dtype: int64

In [5]:
print('Model grid:\n',model.grid)
print('Transition probabilities conditional on no contraception:\n',model.P1)
print('Transition probabilities conditional on contraception:\n',model.P2)
print('Bellman one run:\n',ev)
print('Bellman pk:\n',pk)

Model grid:
 [0 1 2 3 4]
Transition probabilities conditional on no contraception:
 [[0.3 0.7 0.  0.  0. ]
 [0.  0.3 0.7 0.  0. ]
 [0.  0.  0.3 0.7 0. ]
 [0.  0.  0.  0.3 0.7]
 [0.  0.  0.  0.  1. ]]
Transition probabilities conditional on contraception:
 [[0.97 0.03 0.   0.   0.  ]
 [0.   0.97 0.03 0.   0.  ]
 [0.   0.   0.97 0.03 0.  ]
 [0.   0.   0.   0.97 0.03]
 [0.   0.   0.   0.   1.  ]]
Bellman one run:
 [46.21744697 45.73002309 44.90373094 44.04893932 43.49461016]
Bellman pk:
 [0.44934729 0.394585   0.39007826 0.438393   0.52996405]


Estimate parameters with log likelihood loop

In [ ]:
# Find tha likelihood value for different combinations 
ev = np.zeros((model.n))
par_mu = copy.copy(model.mu)
par_eta2 = copy.copy(model.eta2)


NRC = 200
Nc = 200

log_lik = np.nan + np.zeros((NRC,Nc))
mu = np.linspace(-1,1,NRC)
eta2 = np.linspace(-1,1,Nc)

for i in range(NRC):
    for j in range(Nc):
       
        # STEP 1: Find p
        data0 = data[data['d'] == 0]
        data1 = data[data['d'] == 1]
    
        tabulate0 = data0.dx1.value_counts()
        tabulate1 = data1.dx1.value_counts()

        p1 = [tabulate0[i]/sum(tabulate0) for i in range(tabulate0.size)]
        p2 = [tabulate1[i]/sum(tabulate1) for i in range(tabulate1.size)] 
    
        
        
        # STEP 2: Estimate structual parameters
        model.p1 = p1 # Use first step estimates as starting values for t
        model.p2 = p2
        
        # Estimate RC and C
        pnames = ['mu','eta2']
        theta = np.array([mu[i], eta2[j]])
        log_lik[i,j]=estimate.ll(theta,model, solver,data, pnames, no_guess=True)

log_lik *= samplesize*(-1)

#Re-inset the true parameters 
model.mu = copy.copy(par_mu)
model.eta2 = copy.copy(par_eta2)

In [ ]:
# plot figure in three dimensions
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.style.use('seaborn-whitegrid')


fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1,projection='3d', computed_zorder=False)

# Make data.
X, Y = np.meshgrid(mu, eta2,indexing='ij')
x, y = np.unravel_index(np.argmax(log_lik), log_lik.shape)

# Plot the surface.
surf = ax.plot_surface(X, Y, log_lik, cmap=cm.jet)

#Plot max value
max = ax.scatter(mu[x], eta2[y], log_lik[x,y], color=['black'], marker='o', s=10)

# Customize the axis.
ax.set_xlabel(f'mu')
ax.set_ylabel(f'eta2')
ax.set_title(f'Log-likelihood (mu,eta2)')
ax.invert_xaxis()

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()
print(mu[x], eta2[y])


Use estimate function to estimate parameters

In [6]:
theta0 = [0.000001, 0.000001, 0.0000001]
nfxp_model, nfxp_results, pnames, theta_hat, Avar, converged=estimate.estimate(model, solver,data)
# Print the result
print(f'Structual estimation')
print(f'Beta        = {model.beta:.4f}')
print(f'n           = {model.n}')
print(f'Sample size = {samplesize}\n \n')

print(f'Parameters     Estimates    s.e. ') 
print(f'{pnames[0]}              {theta_hat[0]:.4f}     {np.sqrt(Avar[0,0]):.4f} ')
print(f'{pnames[1]}              {theta_hat[1]:.4f}     {np.sqrt(Avar[1,1]):.4f} ')
print(f'{pnames[2]}              {theta_hat[2]:.4f}     {np.sqrt(Avar[2,2]):.4f}  \n ')


print(f'Log-likelihood {-nfxp_results.fun*samplesize:.2f}') 
print(f'The model converged: {converged}')

print(nfxp_results)

Structual estimation
Beta        = 0.9900
n           = 5
Sample size = 20000
 

Parameters     Estimates    s.e. 
mu              0.0947     0.0225 
eta2              0.1260     0.0186 
eta3              0.0443     0.0480  
 
Log-likelihood -20645.41
The model converged: True
     fun: 1.0322706062581222
    hess: array([[ 0.32100804,  0.97840856, -0.43257223],
       [ 0.97840856,  4.87011834, -2.02005263],
       [-0.43257223, -2.02005263,  0.8653635 ]])
     jac: array([-2.45627589e-08,  6.54765318e-09, -2.02374687e-08])
 message: 'A bad approximation caused failure to predict improvement.'
    nfev: 1216
    nhev: 854
     nit: 1214
    njev: 854
  status: 2
 success: False
       x: array([0.09470205, 0.12596005, 0.04429866])


Estimate parameters on real data

In [ ]:
model=child_model()
solver = solve_NFXP()

In [ ]:
dta = model.read_data()
samplesize = dta.shape[0]
uncond_R_P = sum(dta.d)/samplesize

# Estimate
nfxp_model, optim_res, pnames, theta_hat, Avar, converged=estimate.estimate(model, solver,dta)

In [ ]:
# Print the result
print(f'Structual estimation')
print(f'Beta        = {model.beta:.4f}')
print(f'n           = {model.n}')
print(f'Sample size = {samplesize}\n \n')

print(f'Parameters     Estimates    s.e. ') 
print(f'{pnames[0]}              {theta_hat[0]:.4f}     {np.sqrt(Avar[0,0]):.4f} ')
print(f'{pnames[1]}              {theta_hat[1]:.4f}     {np.sqrt(Avar[1,1]):.4f} ')
print(f'{pnames[2]}              {theta_hat[2]:.4f}     {np.sqrt(Avar[2,2]):.4f}  \n ')


print(f'Log-likelihood {-optim_res.fun*samplesize:.2f}') 
print(f'The model converged: {converged}')
optim_res
print(optim_res)

Code for data processing

In [ ]:
with open('carro-mira.csv', 'r') as file:
    lines = file.readlines()

# Remove the first character from each line
modified_lines = [line[1:] for line in lines]

# Open the output file in write mode
with open('carro-mira.csv', 'w') as file:
    file.writelines(modified_lines)